In [1]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .config("spark.driver.memory","32G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.1.0")\
    .config("spark.kryoserializer.buffer.max", "1000M")\
    .getOrCreate()


:: loading settings :: url = jar:file:/home/3147567/.conda/envs/reddit_env/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/3147567/.ivy2/cache
The jars for the packages stored in: /home/3147567/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea6c0acf-bf57-4082-9a37-cc1afbd7059c;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.1.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

In [3]:
spark.version

'3.1.2'

In [13]:
NeutralFile = spark.read.parquet("../../Files/Submissions/score/done/Neutr_vacc.parquet")
ProFile = spark.read.parquet("../../Files/Submissions/score/done/Pro_vacc.parquet")
AntiFile = spark.read.parquet("../../Files/Submissions/score/done/Anti_vacc.parquet")

In [14]:
ProFile.count()

196185

In [17]:
Total.count()

2982155

In [15]:
import functools
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [16]:
Total = unionAll([NeutralFile, ProFile, AntiFile])

In [15]:
sample = Total.sampleBy("pred_1", fractions={
    0.0: 0.10,
    1.0: 0.10,
    2.0: 0.10
}, seed=42)

In [20]:
sample_p = ProFile

In [57]:
sample_n = NeutralFile

In [59]:
sample_a = AntiFile

In [60]:
sample_a.count()

421328

In [18]:
from sparknlp.base.document_assembler import DocumentAssembler
from sparknlp.base.finisher import Finisher
from sparknlp.annotator.stop_words_cleaner import StopWordsCleaner
from sparknlp.annotator.normalizer import Normalizer
from sparknlp.annotator.token import Tokenizer
from pyspark.ml.clustering import LDA

In [32]:

# remove stopwords
document_assembler = DocumentAssembler() \
    .setInputCol("cleanText") \
    .setOutputCol("document") \
    .setCleanupMode("disabled")
# Split sentence to tokens(array)
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
# clean unwanted characters and garbage
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized") \
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

finisher = Finisher() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

nlp_pipeline = Pipeline(
    stages=[
        document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,  
            finisher])

In [61]:
# train the pipeline
nlp_model = nlp_pipeline.fit(sample_a)

In [62]:
# apply the pipeline to transform dataframe.
processed_df  = nlp_model.transform(sample_a)

In [63]:
tokens_df = processed_df.select('class_II','tokens')
tokens_df.count()

421328

In [64]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=500, minDF=3.0)
# train the model
cv_model = cv.fit(tokens_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)

In [55]:
topic_range = [2,3,4,5,6,7,8]

In [51]:
results = ''
results += "tyhata"
results += "\n"
results += "eesdfse"

In [54]:
print(results)

tyhata
eesdfse


In [80]:
print(results)

*************************
ANTI VACCINES, NUMBER OF TOPICS: 2*************************
The lower bound on the log likelihood of the entire corpus: -44790377.34381053
The upper bound on perplexity: 5.791347903057161

topic: 0
*************************
NUM
vaccine
people
covid
m
get
like
think
know
death
year
time
go
work
die
vaccinate
want
virus
url
say
test
life
day
tell
thing
good
come
ve
world
try
topic: 1
*************************
people
like
m
know
think
want
go
say
time
thing
NUM
need
feel
way
work
get
good
right
ve
look
life
bad
post
remove
man
come
mask
tell
make
fuck
*************************
ANTI VACCINES, NUMBER OF TOPICS: 3*************************
The lower bound on the log likelihood of the entire corpus: -45001099.28986275
The upper bound on perplexity: 5.818594025388976

topic: 0
*************************
vaccine
NUM
m
covid
people
get
know
like
work
go
time
want
year
think
tell
say
ve
day
feel
vaccinate
try
come
take
life
thing
start
good
job
die
talk
topic: 1
**********

In [76]:
results = ''
for topic_n in topic_range:
    num_topics = topic_n
    lda = LDA(k=num_topics, maxIter=10)
    model4 = lda.fit(vectorized_tokens)
    ll = model4.logLikelihood(vectorized_tokens)
    lp = model4.logPerplexity(vectorized_tokens)
    results += "*"*25
    results += "\n"
    results += f"ANTI VACCINES, NUMBER OF TOPICS: {num_topics}"
    results += "*"*25
    results += "\n"
    results += ("The lower bound on the log likelihood of the entire corpus: " + str(ll))
    results += "\n"
    results +=("The upper bound on perplexity: " + str(lp))
    results += "\n"
    results += "\n"
    vocab = cv_model.vocabulary
    topics = model4.describeTopics(maxTermsPerTopic = 30)   
    topics_rdd = topics.rdd
    topics_words = topics_rdd\
        .map(lambda row: row['termIndices'])\
        .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
        .collect()
    for idx, topic in enumerate(topics_words):
        results += ("topic: {}".format(idx))
        results += "\n"
        results += ("*"*25)
        results += "\n"
        for word in topic:
            results += word
            results += "\n"
            # results += ("*"*25)
            # results += "\n"
#write results to file
with open("../../Files/models/topic_an.txt", "w") as output:
    output.write(results)


KeyboardInterrupt: 

In [81]:
with open("../../Files/models/topic_an.txt", "w") as output:
    output.write(results)

In [38]:
# extract vocabulary from CountVectorizer
vocab = cv_model.vocabulary
topics = model4.describeTopics(maxTermsPerTopic = 30)   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
vaccine
covidnum
NUM
covid
remove
booster
pfizer
say
vaccination
million
dose
vaccinate
shot
coronavirus
biden
moderna
get
antivaxxer
jab
delete
uk
covidNUM
eu
trump
fda
americans
mandatory
india
plan
company
*************************
topic: 1
*************************
vaccine
covidNUM
vaccination
world
information
health
NUM
yes
join
global
canada
daily
passport
rule
num
ask
question
case
ampnbsp
discussion
cdc
appointment
find
united
report
answer
texas
concern
approve
proof
*************************
topic: 2
*************************
NUM
day
feel
m
like
get
go
vaccine
time
work
shot
week
experience
ve
know
take
hour
dose
think
num
bad
symptom
start
want
help
pain
second
good
well
thank
*************************
topic: 3
*************************
NUM
vaccine
new
year
covid
dose
love
great
cute
get
amazing
beautiful
m
vaccinate
old
immunity
start
day
say
shot
time
people
pfizer
free
month
school
week
york
coronavirus
gift
*************************
to

In [53]:
model4.save('../../Files/models/topic_a_7.pickle')

In [130]:
model2.save('../../Files/models/topic_a_all.pickle')

In [151]:
model3.save('../../Files/models/topic_p_n.pickle')